# Evaluation Strategies in Operational Semantics

The operational semantics of a language specifies the way(s) in which expressions in that language can be evaluated and the way(s) in which statements in that language (if they exist) can be executed. There are a few common ways in which the operational semantics can deal with evaluation of expressions, and we will call these *evaluation strategies*. In these notes we will review two common evaluation strategies, illustrate these evaluation strategies using Python, and demonstrate how evaluation strategies can play a role in an embedded language.

Whether a language uses an evaluation strategy that is eager or lazy is related to whether that language exports to users a *declarative paradigm*. A declarative programming approach involves defining structures and describing transformations on those structures in an equational way. Using a declarative approach to implement a solution can have a number of benefits:

* the implementation is likely to be concise;
* the implementation may be easier to quickly explore trade-offs between optimality and performance; and
* the implementation may be easier to to store and later restart partial solutions.

## Dependencies

In [16]:
# Presentation dependencies.
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mp
import matplotlib.pyplot as plt
from importlib import reload
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

# Content dependencies (also reproduced inline).
from random import randint
from itertools import permutations
from functools import reduce
from tqdm import tqdm
import numpy as np

## Types of Evaluation Strategies

### Strict Evaluation

When an operational semantics employs a <i>strict evaluation strategy</i> (also known as an <i>eager evaluation strategy</i>), expressions are always evaluated completely before they (or, more precisely, the results of their evaluation) can be employed as an argument within another function. This is the most common evaluation strategy found in programming languages, and the one that is likely most familiar to most audiences. Approaches such as <i>call-by-value</i> are examples of strict evaluation strategies.

In [23]:
# The following Python code is evaluated using
# a strict evaluation strategy.
def print_then_return_three():
    print("This function has been evaluated.")
    return 3

def f(x, y):
    return x

# The second argument is not used, but it is evaluated anyway.
f(2, print_then_return_three())

This function has been evaluated.


2

### Non-strict Evaluation

When an operational semantics employs a <i>non-strict evaluation strategy</i> (also known as a <i>lazy evaluation strategy</i>), expressions are evaluated only if they are used (e.g., the expression corresponding to a function argument is only evaluated if that argument is used somewhere within the body).

While the Python operational semantics definition does not employ a lazy evaluation strategy by default, Python does allow programmers to utilize a lazy evaluation strategy in a number of ways.

Because Python supports the functional programming paradigm (and, in particular, higher-order function and anonymous functions), it is possible to effectively implement lazy evaluation strategies. However, this may be somewhat cumbersome.

In [27]:
# The following Python code is evaluated using
# a strict evaluation strategy.
def make_lazy_number(n):
    
    def lazy_number():
        print("The number " + str(n) + " has been evaluated.")
        return n

    return lazy_number

def f(x, y):
    return x()

# Notice that the second argument is not evaluated.
f(make_lazy_number(2), make_lazy_number(3))

The number 2 has been evaluated.


2

Another way in which Python provides native support for lazy evaluation is via <i>iterators</i>. Python prescribes the use of iterators in this way and makes it convenient to use them via both dedicated syntactic constructs and special-purpose libraries. However, we do note that it is technically possible to implement iterators using lambda abstractions (because the lambda calculus is Turing-complete).

In [1]:
xs = (x for x in range(0,1000000000000))
print(next(xs))
print(next(xs))
print(next(xs))

0
1
2


In the two case studies in these notes, we review how Python iterators can be used to employ lazy evaluation in order to solve optimization problems using a declarative programming approach.

### Non-deterministic Evaluation

Under <i>non-deterministic</i> evaluation strategies, expressions may be evaluated before they are needed (and their evaluation may be aborted if it is discovered they are no longer needed). JavaScript Promises are arguably one example of such a semantics embedded within a programming language as a library.

## Case Study: Shortest Common Superstring

**Definition:** Suppose we are given a collection of $n$ strings (repeats are permitted):

$$S = [s_1, ..., s_n]$$

For two strings $s$ and $r$, let $s \preceq r$ denote that $s$ is a substring of $t$. The *shortest common superstring* of $S$ is the shortest string $t$ such that for all $s_i$ in $S$, $s_i \preceq r$.

We build some collections of randomly generated strings that we will use for our examples.

In [4]:
from random import choice, randint

strings100 = ["".join([choice('abcd') for _ in range(randint(2,4))]) for _ in range(100)]
strings10 = ["".join([choice('abcd') for _ in range(randint(2,5))]) for _ in range(10)]
strings9 = ["".join([choice('abcd') for _ in range(randint(2,5))]) for _ in range(9)]
strings4 = ["".join([choice('abcd') for _ in range(randint(2,5))]) for _ in range(4)]
strings4

['bbcb', 'daa', 'cca', 'cbc']

We define two useful helper functions: a function to generate all possible ways of splitting a string and a function to merge two strings such that the overlapping portion is not duplicated (if it exists).

In [5]:
def splits(s):
    return list(reversed([(s[:i], s[i:]) for i in range(len(s)+1)]))

def merge(s, t):
    return min([s+tr for (tl,tr) in splits(t) if s.endswith(tl)], key=len)

(splits('abcdefg'), merge('abcd', 'cde'))

([('abcdefg', ''),
  ('abcdef', 'g'),
  ('abcde', 'fg'),
  ('abcd', 'efg'),
  ('abc', 'defg'),
  ('ab', 'cdefg'),
  ('a', 'bcdefg'),
  ('', 'abcdefg')],
 'abcde')

### Solution Approach: Recursion

Another way that we can implement an algorithm that finds the optimal solution is using recursion.

In [6]:
# Take out the ith entry and merge it with the input r.
def pick(i, ss, r):
    return (ss[:i]+ss[i+1:], merge(r, ss[i]))

# Generate a nested list (essentially a tree).
def search(ss, r = ''):
    if ss == []:
        return [r]
    else:
        options = [pick(i, ss, r) for i in range(len(ss))]
        results = [search(*o) for o in options]
        return results

# All possible solutions in a nested tree-like data structure.
search(strings4)

[[[[['bbcbdaaccacbc']], [['bbcbdaacbcca']]],
  [[['bbcbccadaacbc']], [['bbcbccacbcdaa']]],
  [[['bbcbcdaacca']], [['bbcbccadaa']]]],
 [[[['daabbcbccacbc']], [['daabbcbcca']]],
  [[['daaccabbcbc']], [['daaccacbcbbcb']]],
  [[['daacbcbbcbcca']], [['daacbccabbcb']]]],
 [[[['ccabbcbdaacbc']], [['ccabbcbcdaa']]],
  [[['ccadaabbcbc']], [['ccadaacbcbbcb']]],
  [[['ccacbcbbcbdaa']], [['ccacbcdaabbcb']]]],
 [[[['cbcbbcbdaacca']], [['cbcbbcbccadaa']]],
  [[['cbcdaabbcbcca']], [['cbcdaaccabbcb']]],
  [[['cbccabbcbdaa']], [['cbccadaabbcb']]]]]

In [7]:
# Find the shortest entry using a tree traversal.
def search(ss, r = ''):
    if ss == []:
        return r
    else:
        options = [pick(i, ss, r) for i in range(len(ss))]
        results = [search(*o) for o in options]
        return min(results, key=len)

# The shortest solution.
search(strings9)

'cbcaadddcddadabbaabb'

One disadvantage of the above is that it is somewhat opaque. While the algorithm is running, there is no feedback on progress. We can address this, however, by building a *generator* using recursion (rather than returning the result). To see how this can be done, first consider how we would use the same recursive approach to build a flattened list of all possible solutions.

In [8]:
# Generate a flattened list.
def search(ss, r = ''):
    if ss == []:
        return [r]
    else:
        options = [pick(i, ss, r) for i in range(len(ss))]
        results = [r for o in options for r in search(*o)]
        return results

# All the solutions.
search(strings4)

['bbcbdaaccacbc',
 'bbcbdaacbcca',
 'bbcbccadaacbc',
 'bbcbccacbcdaa',
 'bbcbcdaacca',
 'bbcbccadaa',
 'daabbcbccacbc',
 'daabbcbcca',
 'daaccabbcbc',
 'daaccacbcbbcb',
 'daacbcbbcbcca',
 'daacbccabbcb',
 'ccabbcbdaacbc',
 'ccabbcbcdaa',
 'ccadaabbcbc',
 'ccadaacbcbbcb',
 'ccacbcbbcbdaa',
 'ccacbcdaabbcb',
 'cbcbbcbdaacca',
 'cbcbbcbccadaa',
 'cbcdaabbcbcca',
 'cbcdaaccabbcb',
 'cbccabbcbdaa',
 'cbccadaabbcb']

We can then convert the above by replacing the list comprehensions with generator comprehensions. Using this solution, it is possile to iterate over all the solutions built using the recursive tree-like traversal (and, for example, to pick the shortest one).

In [9]:
# Build a generator.
def search(ss, r = ''):
    if ss == []:
        return (r for _ in range(1))
    else:
        options = (pick(i, ss, r) for i in range(len(ss)))
        results = (r for o in options for r in search(*o))
        return results

# A generator for solutions.
rs = search(strings4)

# Iterate over the first 10 entries.
for (_, r) in zip(range(10), rs):
    print(r)
    
# Find the shortest solution in the first 10 entries.
from itertools import islice
min(islice(rs, 10), key=len)

bbcbdaaccacbc
bbcbdaacbcca
bbcbccadaacbc
bbcbccacbcdaa
bbcbcdaacca
bbcbccadaa
daabbcbccacbc
daabbcbcca
daaccabbcbc
daaccacbcbbcb


'ccabbcbcdaa'

We can add randomization to the above solution by taking a permutation of the index list of strings at each level of the recursion.

In [17]:
# Build a generator.
def search(ss, r = ''):
    if ss == []:
        return (r for _ in range(1))
    else:
        options = (pick(i, ss, r) for i in np.random.permutation(range(len(ss))))
        results = (r for o in options for r in search(*o))
        return results

# A generator for solutions.
list(search(strings4))

['ccacbcdaabbcb',
 'ccacbcbbcbdaa',
 'ccabbcbcdaa',
 'ccabbcbdaacbc',
 'ccadaabbcbc',
 'ccadaacbcbbcb',
 'cbcdaabbcbcca',
 'cbcdaaccabbcb',
 'cbcbbcbccadaa',
 'cbcbbcbdaacca',
 'cbccabbcbdaa',
 'cbccadaabbcb',
 'bbcbccadaacbc',
 'bbcbccacbcdaa',
 'bbcbcdaacca',
 'bbcbccadaa',
 'bbcbdaacbcca',
 'bbcbdaaccacbc',
 'daaccabbcbc',
 'daaccacbcbbcb',
 'daacbccabbcb',
 'daacbcbbcbcca',
 'daabbcbcca',
 'daabbcbccacbc']

If we want to turn a generator into multiple generators, we can do so by wrapping it in generators that drop some of the values.

In [3]:
from timeit import default_timer
import mr4mp

g = range(10000000)

def mapper(g):
    return max([r//2 for r in g])

def reducer(r1, r2):
    return max([r1, r2])

start = default_timer()

pool = mr4mp.pool(1)
result = pool.mapreduce(mapper, reducer, [g])

print("Finished in " + str(default_timer()-start) + "s using " + str(len(pool)) + " process(es).")
result

Finished in 1.8926288220100105s using 1 process(es).


4999999

In [ ]:
from itertools import islice

g = range(10000000)

g1 = islice(g, 0, None, 4)
g2 = islice(g, 1, None, 4)
g3 = islice(g, 2, None, 4)
g4 = islice(g, 3, None, 4)

gs = [islice(g, start, None, 4) for start in range(0,4)]

def mapper(g):
    return max([r//2 for r in g])

def reducer(r1, r2):
    return max([r1, r2])

start = default_timer()

pool = mr4mp.pool(4)
result = pool.mapreduce(mapper, reducer, gs)

print("Finished in " + str(default_timer()-start) + "s using " + str(len(pool)) + " process(es).")
result

## Case Study: Tic-tac-toe

In [11]:
from parts import parts
from itertools import count

class Board():
    def __init__(self, cells = ['_']*9):
        self.cells = cells
    
    def __repr__(self):
        return "\n"+"\n".join(tuple(" ".join(row) for row in parts(self.cells,3)))+"\n\n"

    def move(self, p, j):
        return Board([p if k==j else c for (c,k) in zip(self.cells, count())])

    def moves(self, p):
        return [self.move(p,k) for (c,k) in zip(self.cells, count()) if c == '_']

    def win(self, p):
        rs = list(map(tuple, parts(self.cells, 3)))
        return (p,p,p) in rs or\
               (p,p,p) in map(tuple, zip(*rs)) or\
               (p,p,p) in [(rs[0][0], rs[1][1], rs[2][2]), rs[0][2], rs[1][1], rs[2][0]]

    def end(self):
        return any([self.win('X'), self.win('O')])

    def __len__(self):
        return self.cells.count('_')
    
    def __eq__(self, other):
        return tuple(self.cells) == tuple(other.cells)
    
b = Board()
b = b.move('X', 1).move('O', 2)
b.moves('X')

[
 X X O
 _ _ _
 _ _ _
 , 
 _ X O
 X _ _
 _ _ _
 , 
 _ X O
 _ X _
 _ _ _
 , 
 _ X O
 _ _ X
 _ _ _
 , 
 _ X O
 _ _ _
 X _ _
 , 
 _ X O
 _ _ _
 _ X _
 , 
 _ X O
 _ _ _
 _ _ X
 ]

In [12]:
from itertools import islice

b = Board()

def search(b, p = 'X'):
    if b.end():
        return [b]
    else:
        return [b for m in b.moves(p) for b in search(m, 'O' if p=='X' else 'X')]

search(b)

[
 X O X
 O X O
 X O X
 , 
 X O X
 O X O
 O X X
 , 
 X O X
 O X O
 _ _ X
 , 
 X O X
 O X X
 O O X
 , 
 X O X
 O X O
 O X X
 , 
 X O X
 O X _
 O _ X
 , 
 X O X
 O X X
 O O X
 , 
 X O X
 O X O
 X O X
 , 
 X O X
 O X _
 _ O X
 , 
 X O X
 O O X
 X O _
 , 
 X O X
 O O X
 O X X
 , 
 X O X
 O O X
 _ _ X
 , 
 X O X
 O X X
 O O X
 , 
 X O X
 O O X
 O X X
 , 
 X O X
 O _ X
 O _ X
 , 
 X O X
 O X X
 O O X
 , 
 X O X
 O O X
 X O _
 , 
 X O X
 O _ X
 _ O X
 , 
 X O X
 O O X
 X O _
 , 
 X O X
 O O O
 X X _
 , 
 X O X
 O O O
 X _ X
 , 
 X O X
 O O _
 X O X
 , 
 X O X
 O X O
 X O X
 , 
 X O X
 O O O
 X X _
 , 
 X O X
 O O O
 X _ X
 , 
 X O X
 O X O
 X O X
 , 
 X O X
 O X O
 X O X
 , 
 X O X
 O O X
 X O _
 , 
 X O X
 O O _
 X O X
 , 
 X O X
 O X O
 X O X
 , 
 X O X
 O O X
 O X X
 , 
 X O X
 O O O
 X X _
 , 
 X O X
 O O O
 _ X X
 , 
 X O X
 O O X
 O X X
 , 
 X O X
 O X O
 O X X
 , 
 X O X
 O O O
 X X _
 , 
 X O X
 O O O
 _ X X
 , 
 X O X
 O X O
 O X X
 , 
 X O X
 O X O
 O X X
 , 
 X O X
 O O X
 O X X
 ,

In [13]:
b = Board()

def gen(b, p = 'X'):
    if b.end():
        return (b for _ in range(1))
    else:
        return (b for m in b.moves(p) for b in gen(m, 'O' if p=='X' else 'X'))

len(list(gen(b)))

182988

In [14]:
from itertools import islice

turns = (p for _ in count() for p in ['X', 'O'])
print(next(turns))
print(next(turns))
print(next(turns))

X
O
X


In [15]:
from itertools import islice

turns = (p for _ in count() for p in ['X', 'O'])
print(list(islice(turns, 4)))

turn_gs = ((p for _ in count() for p in ['X', 'O']) for _ in count())

b = Board()

def gen(b, turn_gs):
    ts = next(turn_gs)
    p = next(ts)
    turn_gs = (islice(ts, 1, None) for ts in turn_gs)
    if len(b.moves(p)) == 7:
        return [b]
    else:
        ms = b.moves(p)
        return (mb for m in ms for mb in gen(m, turn_gs))

list(gen(b, turn_gs))

['X', 'O', 'X', 'O']


[
 X O _
 _ _ _
 _ _ _
 , 
 X _ O
 _ _ _
 _ _ _
 , 
 X _ _
 O _ _
 _ _ _
 , 
 X _ _
 _ O _
 _ _ _
 , 
 X _ _
 _ _ O
 _ _ _
 , 
 X _ _
 _ _ _
 O _ _
 , 
 X _ _
 _ _ _
 _ O _
 , 
 X _ _
 _ _ _
 _ _ O
 , 
 O X _
 _ _ _
 _ _ _
 , 
 _ X O
 _ _ _
 _ _ _
 , 
 _ X _
 O _ _
 _ _ _
 , 
 _ X _
 _ O _
 _ _ _
 , 
 _ X _
 _ _ O
 _ _ _
 , 
 _ X _
 _ _ _
 O _ _
 , 
 _ X _
 _ _ _
 _ O _
 , 
 _ X _
 _ _ _
 _ _ O
 , 
 O _ X
 _ _ _
 _ _ _
 , 
 _ O X
 _ _ _
 _ _ _
 , 
 _ _ X
 O _ _
 _ _ _
 , 
 _ _ X
 _ O _
 _ _ _
 , 
 _ _ X
 _ _ O
 _ _ _
 , 
 _ _ X
 _ _ _
 O _ _
 , 
 _ _ X
 _ _ _
 _ O _
 , 
 _ _ X
 _ _ _
 _ _ O
 , 
 O _ _
 X _ _
 _ _ _
 , 
 _ O _
 X _ _
 _ _ _
 , 
 _ _ O
 X _ _
 _ _ _
 , 
 _ _ _
 X O _
 _ _ _
 , 
 _ _ _
 X _ O
 _ _ _
 , 
 _ _ _
 X _ _
 O _ _
 , 
 _ _ _
 X _ _
 _ O _
 , 
 _ _ _
 X _ _
 _ _ O
 , 
 O _ _
 _ X _
 _ _ _
 , 
 _ O _
 _ X _
 _ _ _
 , 
 _ _ O
 _ X _
 _ _ _
 , 
 _ _ _
 O X _
 _ _ _
 , 
 _ _ _
 _ X O
 _ _ _
 , 
 _ _ _
 _ X _
 O _ _
 , 
 _ _ _
 _ X _
 _ O _
 , 
 _ _ _
 _ X _
 _ _ O
 ,

This is the end of the article.